# 容器化技术Docker
通常是使用Docker开发的
## 安装
![[Pasted image 20240111154022.png]]

## 构建容器
![[Pasted image 20240111155624.png]]
```
#在当前路径下，以Dockerfile_python为File，构建名称为ictp-ap-python的Image
$sudo docker build -t ictp-ap-python -f Dockerfile_python . 
#在当前路径下寻找Dockerfile文件
```
```
#编辑docker file
$vim hello-world1
$sudo docker build -t hello-world1-container -f hello-world1 . 

#启动容器
$sudo docker container run hello-world1

#进入容器 -it 并调用bash编辑器
$sudo docker run -it hello-world1 /bin/bash
$sudo docker run -it ubuntu /bin/bash
```
换源
{"registry-mirrors": ["https://registry.hub.docker.com","http://hub-mirror.c.163.com","https://docker.mirrors.ustc.edu.cn","https://registry.docker-cn.com"]
}

## A unified Dockerfile for Python/Jupyter

```
FROM ubuntu:20.04
LABEL MAINTAINER="He Wang<hewang@ucas.ac.cn>"
ARG RELEASE_NAME=focal
ARG ROOT_PASSWD=root
ENV SSH_PORT 22
ENV PATH /root/miniconda3/bin:$PATH
# Open port
EXPOSE 8888
EXPOSE 22
#
# UTF-8 encoding to support chs characters
#
RUN echo "export LANG=C.UTF-8" >>/etc/profile \
#
# apt sources backup
#
&& cp /etc/apt/sources.list /etc/apt/sources.list.bak \
#
# switch apt source to TUNA
#
&& echo "deb http://mirrors.tuna.tsinghua.edu.cn/ubuntu/ release-name main restricted universe multiverse\n\
deb http://mirrors.tuna.tsinghua.edu.cn/ubuntu/ release-name-updates main restricted universe multiverse\n\
deb http://mirrors.tuna.tsinghua.edu.cn/ubuntu/ release-name-backports main restricted universe multiverse\n\
deb http://mirrors.tuna.tsinghua.edu.cn/ubuntu/ release-name-security main restricted universe multiverse" >/etc/apt/sources.list \
&& sed -i 's/release-name/'$RELEASE_NAME'/g' /etc/apt/sources.list \
#
# remove nvidia sources
#
# && rm /etc/apt/sources.list.d/cuda.list /etc/apt/sources.list.d/nvidia-ml.list \
#
# install necessary packages
#
&& apt-get update \
&& apt-get install -y --no-install-recommends \
  build-essential \
  apt-utils \
  vim \
  openssh-server \
  net-tools \
  iputils-ping \
  wget \
  curl \
  git \
  iptables \
  bzip2 \
  command-not-found \
  ca-certificates \
#
# download & install miniconda
#
&& wget -q https://mirrors.tuna.tsinghua.edu.cn/anaconda/miniconda/Miniconda3-latest-Linux-x86_64.sh \
&& bash Miniconda3-latest-Linux-x86_64.sh -b \
&& rm Miniconda3-latest-Linux-x86_64.sh \
&& ln -s /root/miniconda3/etc/profile.d/conda.sh /etc/profile.d/conda.sh \
#
# switch conda source to TUNA
#
&& echo "channels:\n\
  - defaults\n\
show_channel_urls: true\n\
default_channels:\n\
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main\n\
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/r\n\
custom_channels:\n\
  conda-forge: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud\n\
  msys2: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud\n\
  bioconda: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud\n\
  menpo: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud\n\
  pytorch: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud" >/root/.condarc \
#
# create conda py39 environment
#
&& /root/miniconda3/bin/conda create -n py39 python=3.9 -y \
#
# switch pip source to TUNA & update pip
#
&& /root/miniconda3/envs/py39/bin/pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple \
&& /root/miniconda3/envs/py39/bin/pip install pip -U \
#
# enable root access with ssh: replace line starts with '#PermitRootLogin' or 'PermitRootLogin'
#
&& sed -i '/^#*PermitRootLogin/cPermitRootLogin yes' /etc/ssh/sshd_config \
#
# set root password
#
&& echo "root:$ROOT_PASSWD" \
&& echo "root:$ROOT_PASSWD" | chpasswd \
#
# change ssh port & start ssh service when docker container starts
#
&& echo "#!/bin/bash\n\
sed -i \"s/Port 22/Port \$SSH_PORT/g\" /etc/ssh/sshd_config\n\
service ssh start\n\
/bin/bash" >./startup.sh \
&& chmod 777 ./startup.sh \
#
# add welcome tips
#
&& CONDA_VERSION=`/root/miniconda3/condabin/conda -V` \
&& CONDA_VERSION=${CONDA_VERSION#* } \
&& echo "\
printf '\\\n'\n\
printf '\\\033[0;34m'\n\
printf 'Welcome to ICTP-AP server\\\n'\n\
printf 'Please change the default root password in time\\\n'\n\
printf '\\\n'\n\
printf ' Conda:\\\t\\\tMiniconda v$CONDA_VERSION  use \`conda activate py39\` to activate conda env\\\n'\n\
printf ' SSH port:\\\t$SSH_PORT for this container\\\n'\n\
printf '\\\033[0m'\n\
printf '\\\n'\n\
" >>/etc/update-motd.d/10-help-text \
#
# cleanup
#
&& rm -rf /var/lib/apt/lists/* /tmp/*
#
# set startup script
#
ENTRYPOINT ./startup.sh
```

## 先build 生成image
```
#!/bin/bash
sudo docker build -t ictp_ap_jupyter . # build名为ictp_ap_jupyter的image， .依然代表自动寻找Dockerfile
```

## run生成的image，生成container，并设置端口
```
#!/bin/bash
sudo docker run -itd \
	-p 1234:22 \ # 本机1234端口和容器22端口SSH对应
	-p 19999:8888 \ # 本机19999端口和容器8888端口SSH对应
	--name ictp_ap_dev \ # 指定conteriner name
	ictp_ap_jupyter # 指定用于生成container的image
```
注意端口重复
## 用SSH登陆访问
```
$ssh -p 1234 root@0.0.0.0
#本机1234端口连入，默认密码root

$conda env list # 检验预安装的conda
```
## 停止和发布
```
$sudo docker stop [containerID]	# 终止容器运行

# 标注用户名和版本，增加了tag
$sudo docker tag [imageName] [username]/[respository]:[tag]

# 发布image文件
$sudo docker image push [username]/[respository]:[tag]
```
如何解决dockerhub的登陆问题？

# Linux 基础操作
```
$command name [params]  [subject]
```

以空格分隔，多个空格视为一个

Tab键：对命令、参数或者文件的内容补全

CTRL+c 终止运行

CTRL+d 键盘停止输入

CTRL+l 清空清屏

## man
调用说明文档

/ 从上向下搜索
? 从下向上搜索
n 定位到下一个
N 定位到上一个
q 退出帮助文档

## 系统工作命令
![[Pasted image 20240110213510.png]]
![[Pasted image 20240110214515.png]]
用户@主机名称:~home目录$root user

## top
置顶的5行显示系统状态
![[Pasted image 20240110214310.png]]

## watch
```
$watch -n 1 -d uptime
```
定时任务监控 1是时间单位秒

## 系统状态检测&查找定位文件
![[Pasted image 20240110215020.png]]
## grep
```$history | grep python```
从命令历史中筛选'python'
## cat
将文件信息打印
```
$cat ~/.bash history
```
## htop
交互式进程查看器
## ls -lht
带有大小和排序的文件列表显示
## 用户身份与文件权限
![[Pasted image 20240110220201.png]]
d开头对应directory，否则-对应文件
![[Pasted image 20240110220213.png]]
## chmod 更改权限
```
$chmod 777 example.sh
```
also see $man chmod

## find
![[Pasted image 20240110221445.png]]
## Linux 文件结构理解
内容是分配到根目录的各个目录下的，而并非在同一目录下分别存放。
回到根目录即$cd ~
使用which或者whereis

## du
```
$du sh ./* | grep G
```
使用通配符查看所有达到G大小的文件

## 文件编辑
![[Pasted image 20240110224506.png]]
```
$head -n 20
```
```
$ls -l | wc -l #以行打印出文件，数出一共有多少行
$ls -l | grep python #以行打印出文件，找出其中包含python的
```
![[Pasted image 20240110225107.png]]

## 管道符、重定向、环境变量
```
$echo hello > hello.txt
$cat < hello.txt > hello3.txt

$echo hello! > hello.txt #覆盖
$echo hello! >> hello.txt #追加

$echo hello! 2>> hello.txt #仅追加报错信息
$echo hello! &>> hello.txt #全部追加
```
![[Pasted image 20240110225736.png]]

## 环境变量
![[Pasted image 20240110230512.png]]
```
$alias llt = 'ls -lht' #别名赋值
$unalias llt

#可以在./bashrc中找到alias存储
```
```
#定义环境变量，通常全大写
$WORKDIR=/home/ubuntu/main
$cd WORKDIR 
$pwd
>>>/home/ubuntu/main
```

## Vim编辑器

Lecture3： Editors (vim) (2020)
《用聪明的方法学习Vim》

## Shell命令脚本
![[Pasted image 20240110231751.png]]
; 是同时执行的

## SSH服务管理远程设备
![[Pasted image 20240110232048.png]]
大量数据传输使用scp，较少使用rsync

![[Pasted image 20240110232252.png]]